In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Cust = set(range(1,4))
DC = Cust.copy() 
Route = {(i,j) for i in DC for j in Cust}

In [3]:
delivery = {(i,j):5 for i,j in Route}
dc_run = {i:1000 for i in DC}

demand = {j:10 for j in Cust}

In [4]:
model = Model()


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [5]:
x, y = {}, {}

for i,j in Route:
    x[i,j] = model.addVar(vtype="C", name=f'x[{i},{j}]')

for i in DC:
    y[i] = model.addVar(vtype="B", name=f'y[{i}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons = {}, {}

for j in Cust:
    Cust_Demand_Cons[j] = model.addConstr(
        quicksum(x[i,j] for i in DC)
        ==
        demand[j]
    )

for i,j in Route:
    DC_Running_Cons[i,j] = model.addConstr(
        x[i,j]
        <=
        demand[j] * y[i]
    )
    
model.update()

In [7]:
model.setObjective(
    quicksum(delivery[i,j] * x[i,j] for i,j in Route) +
    quicksum(dc_run[i] * y[i] for i in DC)
    ,GRB.MINIMIZE
)

model.update()

In [8]:
model.optimize()

Optimize a model with 12 rows, 12 columns and 27 nonzeros
Variable types: 9 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Presolve removed 8 rows and 6 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 9 nonzeros
Variable types: 3 continuous, 3 integer (3 binary)

Root relaxation: objective 1.150000e+03, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1150.0000000 1150.00000  0.00%     -    0s

Explored 0 nodes (3 simplex iterations) in 0.01 seconds
Thread count was 8 (of 8 available processors)

Solution count 1: 1150 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.150000000000e+03, best bound 1.150000000000e+03, gap 0.0000%


In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[2,1] 10.0
x[2,3] 10.0
x[2,2] 10.0
y[2] 1.0
